In [1]:
# Load packages
import os
import pandas as pd
import numpy as np
 # Required for basic python plotting functionality
import matplotlib.pyplot as plt
# Required for formatting dates later in the case
import datetime
import matplotlib.dates as mdates
# Advanced plotting functionality with seaborn
import seaborn as sns

sns.set(style="whitegrid")  # can set style depending on how you'd like it to look

In [2]:
productInfoDF = os.path.join(os.getcwd(), "./products_info.csv")
productInfoDF = pd.read_csv(productInfoDF)

In [18]:
productInfoDF["Primary Essential Function"].unique()

array(['LC - Digital Learning Platforms',
       'LC - Sites, Resources & Reference - Games & Simulations',
       'LC - Courseware & Textbooks', 'LC - Study Tools - Q&A',
       'LC - Sites, Resources & Reference - Streaming Services',
       'LC - Study Tools',
       'CM - Teacher Resources - Professional Learning',
       'SDO - Learning Management Systems (LMS)',
       'LC - Content Creation & Curation',
       'LC - Sites, Resources & Reference',
       'LC - Online Course Providers & Technical Skills Development',
       'CM - Classroom Engagement & Instruction - Communication & Messaging',
       'SDO - School Management Software - SSO',
       'LC - Sites, Resources & Reference - Thesaurus & Dictionary',
       'LC/CM/SDO - Other', 'SDO - Data, Analytics & Reporting',
       'LC - Sites, Resources & Reference - Encyclopedia',
       'CM - Classroom Engagement & Instruction - Classroom Management',
       'LC - Sites, Resources & Reference - Digital Collection & Repository',
 

In [3]:
engagmentListOfFiles = os.listdir("./engagement_data")
engagmentDict = {}
for csv in engagmentListOfFiles:
    engagmentDict[csv[:-4]] = pd.read_csv("./engagement_data/"+csv)

In [4]:
# Get data
districtDF = os.path.join(os.getcwd(), "./districts_info.csv")
districtDF = pd.read_csv(districtDF)

statesOfInterest = districtDF.state.unique()
print(statesOfInterest)
statesOfInterest = list (statesOfInterest)


['Illinois' nan 'Utah' 'Wisconsin' 'North Carolina' 'Missouri'
 'Washington' 'Connecticut' 'Massachusetts' 'New York' 'Indiana'
 'Virginia' 'Ohio' 'New Jersey' 'California' 'District Of Columbia'
 'Minnesota' 'Arizona' 'Texas' 'Tennessee' 'Florida' 'North Dakota'
 'New Hampshire' 'Michigan']


In [5]:
districtDF[districtDF['state'].isna()].isna().sum()


district_id                  0
state                       57
locale                      57
pct_black/hispanic          57
pct_free/reduced            57
county_connections_ratio    57
pp_total_raw                57
dtype: int64

From this we can tell that when the state value is equal to nan all other Nan collumn values are also nan except district_id . We should take a closer look at the engagment data for those districts

In [6]:
listofNanStates = list(districtDF[districtDF['state'].isna()].district_id)
for i in listofNanStates: 
    if i in engagmentDict:
        print(i)
# For the sake of memory i am going to delete the dictionary until i need it
del engagmentDict

This tells us that the district engagement data is not available for all of the rows where state="Nan". We will ommit this data as it has no value to us now and it will be impossible to find which district ID matches the state,locale etc... 

In [7]:
districtDF = districtDF[districtDF['state'].notna()]
statesOfInterest.remove(np.nan)
statesOfInterest= [str.upper(x) for x in statesOfInterest]

<h1> Exploring the Policy DataFrame</h1>

In [8]:
def preProcessingData():
    policyDataFrame = os.path.join(os.getcwd(), "./COVID-19 US state policy 3_29_2021.csv")
    policyDataFrame = pd.read_csv(policyDataFrame)
    originalStateColumn = policyDataFrame["STATE"]
    policyDataFrame = policyDataFrame.T
    dfs = [x for _, x in policyDataFrame.groupby(1)] # this allowed me to make 26 different series so i can take a look at each category carefully 
    listofDfs = []
    for miniDf in dfs: # This put the category description back 
        tempdf = miniDf.T
        tempdf["STATE"] = originalStateColumn
        listofDfs.append(tempdf)
    return listofDfs
    # for k in range (0, len(listofDfs)): # This allowed me to parse through each category dataframe to inspect it without having to save it onto my hard-drive giving me O(1) Space :) 
    #     temp = pd.DataFrame(listofDfs[k].iloc[0])
    #     print(listofDfs[k].T.reset_index()) 
def getMeDataFrame(k :int  , listofDfs : pd.DataFrame):
    temp = pd.DataFrame(listofDfs[k].iloc[0])
    return listofDfs[k].T.reset_index()
    # print(listofDfs[k].T.reset_index()) 


In [9]:
# Notes 
listofDfs = preProcessingData()
arbitraryNum = 22
getMeDataFrame(arbitraryNum,listofDfs) # This allowed me to parse through each category dataframe to inspect it without having to save it onto my hard-drive giving me O(1) Space :) 


,index,0,1,2,3,4,5,6,7,8,...,47,48,49,50,51,52,53,54,55,56
0,CLBAR3,Close Bars (x3),third_closures,start,date,0,0,0,0,0,...,0,0,0,0,0,0,0,0,NaN,NaN
1,CLRST3,Close Indoor Dining (x3),third_closures,start,date,0,0,0,0,0,...,0,0,0,0,0,0,0,0,NaN,NaN
2,END_CLRST3,Reopen Indoor Dining (x3),third_closures,end,date,0,0,0,0,0,...,0,0,0,0,0,0,0,0,NaN,NaN
3,STATE,State,category,type,unit,Alabama,Alaska,Arizona,Arkansas,California,...,Texas,Utah,Vermont,Virginia,Washington,West Virginia,Wisconsin,Wyoming,NaN,NaN


This Preprossing of data allowed us to get these following categories of interest
- Reopening,food_security, masks,physical_distance_closure,physical_distance_closures,pre_covid_policy
- population_density,quarantines,reopening,second_closures,shelter,state_characteristics, State of emergency issued	

Along with These topics within each category : 
- Category [minimum_wage] = MINWAGEJAN2020	MINWAGEJUL2020	MINWAGESEP2020	MINWAGEOCT2020	TIPMINWAGE2020 (currently not focusing on this)
- Category [unemployment] = UIMAXDUR (currently not focusing on this)

<h1>End of pre-processing</h1>

In [10]:
policyDataFrame = os.path.join(os.getcwd(), "./COVID-19 US state policy 3_29_2021.csv")
policyDataFrame = pd.read_csv(policyDataFrame)
categoriesOfInterest = ["category","Reopening","food_security", "masks","physical_distance_closure","physical_distance_closures","pre_covid_policy"
"population_density","quarantines","reopening","second_closures","shelter,state_characteristics", "State of emergency issued"]

In [11]:
policyDataFrame= policyDataFrame.T[policyDataFrame.T[1].isin( categoriesOfInterest)].T
policyDataFrame= policyDataFrame.drop([55, 56])#.T # Drop the last two empty rows
policyDataFrame["STATE"] = policyDataFrame["STATE"].str.upper()

In [12]:
policyDataFrame.head()

,STATE,CLSCHOOL,CLDAYCR,OPNCLDCR,CLNURSHM,CLBSNS,CURFEW,END_BSNS,FM_ALL,FM_ALL2,...,SNAPALLO,SNAPEBT20,SNAPEBT21,SNAPSUSP,SNAPTLW,CASCLOSE,CASOPEN,CASCLOSE2,CASOPEN2,CASTRIBCAS
0,STATE,Date closed K-12 public schools,Closed day cares,Reopen day cares,Date banned visitors to nursing homes,Closed other non-essential businesses,Closed businesses overnight,Began to reopen businesses,Mandate face mask use by all individuals in pu...,Second mandate for facemasks by all individual...,...,SNAP Waiver - Emergency Allotments to Current ...,SNAP Waiver - Pandemic EBT during school year ...,SNAP Waiver - Pandemic EBT during school year ...,SNAP Waiver - Temporary Suspension of Claims C...,2020 Q1 SNAP ABAWD Time Limit Waiver,State-Mandated Casino Closure,State-Mandated Casino Re-Opening,Second Casino Closure,Second Casino Re-Opening,Mention of Tribal Casinos
1,CATEGORY,physical_distance_closure,physical_distance_closure,Reopening,physical_distance_closure,physical_distance_closure,physical_distance_closure,reopening,masks,masks,...,food_security,food_security,food_security,food_security,food_security,physical_distance_closures,reopening,second_closures,second_closures,physical_distance_closures
2,TYPE,start,start,end,start,start,start,end,start,start,...,start,start,start,start,attribute,start,end,start,end,attribute
3,UNIT,date,date,date,date,date,end,date,date,date,...,date,date,date,date,flag,date,date,date,date,flag
4,ALABAMA,20/3/2020,20/3/2020,23/5/2020,19/3/2020,28/3/2020,0,30/4/2020,16/7/2020,0,...,24/3/2020,4/21/2020,0,0,0,28/3/2020,22/5/2020,0,0,0


In [13]:
policyDataFrame['STATE'].unique()

array(['STATE', 'CATEGORY', 'TYPE', 'UNIT', 'ALABAMA', 'ALASKA',
       'ARIZONA', 'ARKANSAS', 'CALIFORNIA', 'COLORADO', 'CONNECTICUT',
       'DELAWARE', 'DISTRICT OF COLUMBIA', 'FLORIDA', 'GEORGIA', 'HAWAII',
       'IDAHO', 'ILLINOIS', 'INDIANA', 'IOWA', 'KANSAS', 'KENTUCKY',
       'LOUISIANA', 'MAINE', 'MARYLAND', 'MASSACHUSETTS', 'MICHIGAN',
       'MINNESOTA', 'MISSISSIPPI', 'MISSOURI', 'MONTANA', 'NEBRASKA',
       'NEVADA', 'NEW HAMPSHIRE', 'NEW JERSEY', 'NEW MEXICO', 'NEW YORK',
       'NORTH CAROLINA', 'NORTH DAKOTA', 'OHIO', 'OKLAHOMA', 'OREGON',
       'PENNSYLVANIA', 'RHODE ISLAND', 'SOUTH CAROLINA', 'SOUTH DAKOTA',
       'TENNESSEE', 'TEXAS', 'UTAH', 'VERMONT', 'VIRGINIA', 'WASHINGTON',
       'WEST VIRGINIA', 'WISCONSIN', 'WYOMING'], dtype=object)

In [14]:
mask = policyDataFrame['STATE'].isin(statesOfInterest)
mask[0:4] = True # keep the first 4 headers 
policyDataFrame = policyDataFrame[mask].reset_index(drop=True)
policyDataFrame.head(10)

,STATE,CLSCHOOL,CLDAYCR,OPNCLDCR,CLNURSHM,CLBSNS,CURFEW,END_BSNS,FM_ALL,FM_ALL2,...,SNAPALLO,SNAPEBT20,SNAPEBT21,SNAPSUSP,SNAPTLW,CASCLOSE,CASOPEN,CASCLOSE2,CASOPEN2,CASTRIBCAS
0,STATE,Date closed K-12 public schools,Closed day cares,Reopen day cares,Date banned visitors to nursing homes,Closed other non-essential businesses,Closed businesses overnight,Began to reopen businesses,Mandate face mask use by all individuals in pu...,Second mandate for facemasks by all individual...,...,SNAP Waiver - Emergency Allotments to Current ...,SNAP Waiver - Pandemic EBT during school year ...,SNAP Waiver - Pandemic EBT during school year ...,SNAP Waiver - Temporary Suspension of Claims C...,2020 Q1 SNAP ABAWD Time Limit Waiver,State-Mandated Casino Closure,State-Mandated Casino Re-Opening,Second Casino Closure,Second Casino Re-Opening,Mention of Tribal Casinos
1,CATEGORY,physical_distance_closure,physical_distance_closure,Reopening,physical_distance_closure,physical_distance_closure,physical_distance_closure,reopening,masks,masks,...,food_security,food_security,food_security,food_security,food_security,physical_distance_closures,reopening,second_closures,second_closures,physical_distance_closures
2,TYPE,start,start,end,start,start,start,end,start,start,...,start,start,start,start,attribute,start,end,start,end,attribute
3,UNIT,date,date,date,date,date,end,date,date,date,...,date,date,date,date,flag,date,date,date,date,flag
4,ARIZONA,16/3/2020,0,0,0,31/3/2020,0,8/5/2020,0,0,...,1/4/2020,4/17/2020,2/25/2021,0,1,0,0,0,0,0
5,CALIFORNIA,23/3/2020,0,0,0,19/3/2020,11/21/2020,8/5/2020,18/6/2020,0,...,30/3/2020,4/23/2020,0,0,1,0,0,0,0,0
6,CONNECTICUT,17/3/2020,0,0,9/3/2020,23/3/2020,11/2/2020,20/5/2020,20/4/2020,0,...,28/3/2020,4/24/2020,0,2/4/2020,1,0,0,0,0,1
7,DISTRICT OF COLUMBIA,16/3/2020,0,0,0,25/3/2020,11/23/2020,29/5/2020,17/4/2020,0,...,1/4/2020,5/19/2020,3/22/2021,0,1,0,0,0,0,0
8,FLORIDA,17/3/2020,0,0,15/3/2020,3/4/2020,0,18/5/2020,0,0,...,30/3/2020,5/27/2020,3/1/2021,0,0,0,0,0,0,0
9,ILLINOIS,17/3/2020,23/3/2020,29/5/2020,0,21/3/2020,0,29/5/2020,1/5/2020,0,...,1/4/2020,4/17/2020,1/15/2021,0,1,16/3/2020,1/7/2020,20/11/2020,0,0


In [17]:
os.makedirs('folder/subfolder', exist_ok=True)  
policyDataFrame.to_csv('folder/subfolder/out.csv') 

### BREAK Nothing ignore everything past this 

In [12]:
# temp = pd.DataFrame(policyDataFrame.columns.tolist())
# temp["Yes/No"] = np.nan
# # print(policyDataFrame.columns.tolist())

In [46]:
temp = policyDataFrame.describe()

In [68]:
# alice_columns = df.loc[:, df.iloc[0] == 'Alice']


In [ ]:
# DifferentCategories = 
pd.DataFrame(policyDataFrame.iloc[1]).groupby(by=1).value_counts()



In [ ]:
pd.DataFrame(DifferentCategories[1].unique())

In [ ]:
Categories_of_interest = [0,1,2,]

In [ ]:
policyDataFrame.loc[:, policyDataFrame.iloc[1] == 'state_of_emergency']



In [ ]:
districtDF.shape
districtDF.head()

In [ ]:
engagmentDF.head()

In [ ]:
productInfoDF.head()

In [ ]:
(districtDF.shape)
print(engagmentDF.shape)
print(productInfoDF.shape)


In [ ]:
stateDisc = districtDF[["state","district_id"]].groupby(["state"])["district_id"].count().to_frame()

In [ ]:
productInfoDF["Sector(s)"].unique()

In [ ]:
temp1 = os.path.join(os.getcwd(), "./engagement_data/1000.csv")
engData1000 = pd.read_csv(temp1)

temp2 = os.path.join(os.getcwd(), "./engagement_data/1039.csv")
engData1039 = pd.read_csv(temp2)

temp3 = os.path.join(os.getcwd(), "./engagement_data/1044.csv") 
engData1044 = pd.read_csv(temp3)

In [ ]:
len(engData1000["time"].unique())

In [ ]:
productInfoDF["Primary Essential Function"].unique()
# LC = Learning & Curriculum,
# CM = Classroom Management, 
# and SDO = School & District Operations. 
# Each of these categories have multiple sub-categories 
# with which the products were labeled |

In [ ]:
temp4 = os.path.join(os.getcwd(), "./engagement_data/8815.csv") 
engData8815 = pd.read_csv(temp4)
len(engData8815["time"].unique())

In [ ]:
DennisDF = pd.read_csv(os.path.join(os.getcwd(), "./ACSST1Y2021.S1501-2023-03-08T022240.csv"))

In [ ]:
print(DennisDF.columns)

In [ ]:
# discritDF[["locale","pct_free/reduced","state"]].groupby(["locale","pct_free/reduced"]).count()
districtDF.groupby(["state","pct_free/reduced"]).count()

Engagement_data 
<p1>The engagement data are aggregated at school district level, and each file in the folder `engagement_data` represents data from one school district. The 4-digit file name represents `district_id` which can be used to link to district information in `district_info.csv`. The `lp_id` can be used to link to product information in `product_info.csv`.
</p1>